# **CFDBench**

#### 1.模型简介
CFDBench 是一个用于评估在具有各种边界条件、物理特性和域几何形状的流体动力学中机器学习方法的大规模基准。它由计算流体动力学（CFD）中的四个经典问题组成，具有许多不同的操作参数，使其非常适合测试替代模型的推理时间泛化能力。
这种泛化能力对于在将替代模型应用于新问题时避免昂贵的重新训练至关重要。

<img src="./figs/flow-examples.png" alt="模型架构图" width="1000" >
    
#### 2.环境安装
本项目环境基于 **Python 3.10**，`DTK 24.04`，`PyTorch 2.0`，`OneScience`，`json`，`typed-argument-parser`等依赖库。

在超算互联网平台上，我们提供了预先配置好的镜像，创建容器后即可获得已经设置好的运行环境，实现一键运行。

首先使用以下指令手动更新安装onescience。

此外还需要安装其他一些依赖库

In [ ]:
#更新onescience
!pip install onescience-0.1.2-py3-none-any.whl
!pip install typed-argument-parser

#### 3.环境依赖检查
环境检测包含pytorch版本和dcu环境检测。

In [6]:
import torch
version = torch.__version__
num = float(version[:3])
assert num >= 2.0, "Pytorch version must >= 2.0"

assert torch.cuda.is_available(), "Pytorch need DCU"

#### 4.素材准备

数据集中包括的四个流动问题，对于每个问题，我们生成具有不同操作参数的流动，这是我们用来指三种条件的组合的术语：(1)边界BC，(2)流体物理性质(PROP)，和(3)场的几何形状(GEO)。每种运行参数对应一个子集。在每个子集中，相应的操作条件是变化的，而其他参数保持不变。
数据使用npy文件存储，是NumPy 使用的标准二进制文件格式。每组数据下包含u.npy和v.npy以及一个json文件（用于描述这组数据对应的物理条件）

在应用商城 https://www.scnet.cn/ui/mall/app 中搜索“DeepCFDdata”数据集即可下载使用。

#### 5.训练模型

将数据集解压后放入当前目录下的`data`文件夹中，就像这样

```
▼ data/
    ▼ cavity/
        ▼ bc/
        ▼ geo/
        ▼ prop/
    ► tube/
    ► dam/
    ► cylinder/
► tube/   
args.py
train_auto.py
train.py
README.md
```
运行`train.py`或`train_auto.py`分别训练非自回归或自回归模型。使用`--model`指定模型，它必须是以下之一：

Model                          | Value for `--model`    | Script
:----------------------------- | :--------------------- | :-------
Non-autoregrssive FFN          | `ffn`                 | `train.py`
Non-autoregressive DeepONet    | `deeponet`            | `train.py`
Autoregressive Auto-FFN        | `auto_ffn`            | `train_auto.py`
Autoregressive Auto-DeepONet   | `auto_deeponet`       | `train_auto.py`
Autoregressive Auto-EDeepONet  | `auto_edeeponet`      | `train_auto.py`
Autoregressive Auto-DeepONetCNN| `auto_deeponet_cnn`   | `train_auto.py`
Autoregressive ResNet          | `resnet`              | `train_auto.py`
Autoregressive U-Net           | `unet`                | `train_auto.py`
Autoregressive FNO             | `fno`                 | `train_auto.py`

运行`python train.py -h`或`python train_auto.py -h`

例如，使用FNO模型模拟方腔流子集中的所有子集：

In [ ]:
!python train_auto.py --model fno --data cavity_prop_bc_geo

使用DeepONet模型模拟坝流中的 PROP + GEO 子集：

In [ ]:
!python train.py --model deeponet --data_name dam_prop_geo

####模型推理

运行`train.py` 或 `train_auto.py`时使用`--mode test`参数，例如

In [ ]:
!python train.py --model ffn --mode test --data_name dam_prop_geo